# Time Series Analysis

Kasus ini akan melakukan time series analysis dari dataset bernama Daily Climate Series.
Link Dataset: https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data

Kasusnya adalah ingin memprediksi cuaca.

Notes: 
Belum ada info diawalain itu memprediksi di satu langkah kedepan (1 hari) atau beberapa langkah kedepan

# Importing Library

In [2]:
import numpy as np
import pandas as pd
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf

/Users/dicoding/Dicoding/github/mlops/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Importing Dataset

In [3]:
df = pd.read_csv("https://github.com/ridhaginanjar/mlops-explore/releases/download/climate-time-series/DailyDelhiClimateTrain.csv")
df.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


# Cleaning Dataset

## Cleansing

In [5]:
# Checking null values
df.isnull().sum()

date            0
meantemp        0
humidity        0
wind_speed      0
meanpressure    0
dtype: int64

## Feature Selection

In [6]:
dates = df['date'].values
temp = df['meantemp'].values

temp

array([10.        ,  7.4       ,  7.16666667, ..., 14.0952381 ,
       15.05263158, 10.        ])

In [12]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1) # Menambah dimensi untuk series -> e.g: [1,2,3,4] menjadi [[1],[2],[3],[4],[5]]
    ds = tf.data.Dataset.from_tensor_slices(series) # Mengubah series menjadi dataset tensorflow
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Melakukan windowing (dalam kasus ini 60 +1)
    ds = ds.flat_map(lambda x:x.batch(window_size +1)) # Melakukan flattening -> e.g: [[1],[2],[3],[4],[5]] menjadi [1,2,3] [2,3,4] [3,4,5]
    ds = ds.shuffle(shuffle_buffer) # Mengacak urutan window -> [2,3,4] [1,2,3] [3,4,5]
    ds = ds.map(lambda x: (x[:-1], x[-1:])) # memecah x dan y -> x = [[2,3]] y =[[4]]
    return ds.batch(batch_size).prefetch(1) # mengelompokkan per batch -> e.g: batch 1 x = [[2,3]] y =[[4]], batch 2 = xxxx 

# Training

In [8]:
train_set = windowed_dataset(temp, window_size=60, batch_size=100, shuffle_buffer=1000)
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(60, return_sequences=True),
    tf.keras.layers.LSTM(60),
    tf.keras.layers.Dense(30, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [10]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1.000e-04, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
history = model.fit(train_set, epochs=100)

Epoch 1/100


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 25.3102 - mae: 25.8102
Epoch 2/100
 4/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 24.3077 - mae: 24.8077

2025-05-09 13:19:31.127555: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/dicoding/Dicoding/github/mlops/.venv/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 25.1145 - mae: 25.6145
Epoch 3/100
 4/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 24.8023 - mae: 25.3023

2025-05-09 13:19:31.546109: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


...

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 21.5065 - mae: 22.0065
Epoch 18/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 20.7455 - mae: 21.2455
Epoch 19/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 19.9046 - mae: 20.4046
Epoch 20/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 18.1369 - mae: 18.6369
Epoch 21/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 16.7430 - mae: 17.2429
Epoch 22/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 14.4109 - mae: 14.9075
Epoch 23/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - loss: 11.1171 - mae: 11.6070
Epoch 24/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 7.9854 - mae: 8.4734
Epoch 25/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.9023 - mae: 6.3919
Epoch 26/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 5.1244 - mae: 5.6136
Epoch 27/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 5.1849 - mae: 5.6629
Epoch 28/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 4.9437 - mae: 5.4229
Epoch 29/100
15

2025-05-09 13:19:44.177321: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 3.6791 - mae: 4.1562
Epoch 34/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3.2002 - mae: 3.6733
Epoch 35/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 3.8480 - mae: 4.3237
Epoch 36/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3.0823 - mae: 3.5598
Epoch 37/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 3.0326 - mae: 3.4958
Epoch 38/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.9789 - mae: 3.4489
Epoch 39/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.1077 - mae: 2.5693
Epoch 40/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 4.5772 - mae: 5.0580
Epoch 41/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 2.5857 - mae: 3.0530
Epoch 42/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 2.0264 - mae: 2.4832
Epoch 43/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 2.0198 - mae: 2.4782
Epoch 44/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.6854 - mae: 2.1379
Epoch 45/100
15/15 ━━━━━━━━━━

2025-05-09 13:19:57.637501: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.2262 - mae: 1.6507
Epoch 66/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.2044 - mae: 1.6399
Epoch 67/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.2506 - mae: 1.6806
Epoch 68/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.1017 - mae: 1.5293
Epoch 69/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.5008 - mae: 1.9513
Epoch 70/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.3094 - mae: 1.7560
Epoch 71/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.1065 - mae: 1.5350
Epoch 72/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.1310 - mae: 1.5564
Epoch 73/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.1615 - mae: 1.5929
Epoch 74/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.3542 - mae: 1.7975
Epoch 75/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.1949 - mae: 1.6257
Epoch 76/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 1.1043 - mae: 1.5314
Epoch 77/100
15/15 ━━━━━━━━━━

# Evaluate

In [11]:
forecast = history.model.predict(train_set)
forecast

15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


/Users/dicoding/Dicoding/github/mlops/.venv/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


array([[17.671978],
       [28.751707],
       [31.614351],
       ...,
       [28.973856],
       [15.422614],
       [18.634449]], dtype=float32)